In [2]:
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [3]:
dataset = 'cityscapes'
# dataset= 'acdc'
# labels_dict_h = {0:'drivable', 1:'flat non-drivable', 2:'obstacle', 3:'road sign', 4:'sky', 5:'human', 6:'vehicle', 7:'abstain'}
# labels_dict = pickle.load(open('/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/data/cityscapes_trainIds.pkl', 'rb'))
# stats = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/agg_stats/agg_stats_99.pkl', 'rb'))
baseline_stats_cs = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/baseline_hs_100.pkl', 'rb'))
stats_cs = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/stats_with_h_99.pkl', 'rb'))
# stats_h = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/agg_stats/stats_h_99.pkl', 'rb'))
dataset = 'acdc'
baseline_stats_acdc = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/baseline_hs_100.pkl', 'rb'))
stats_acdc = pickle.load(open(f'/BS/mlcysec2/work/robust-segmentation/code/hrnet_seg/{dataset}_logs/paper_logs/stats_with_h_2.pkl', 'rb'))

In [4]:
list(stats_acdc.items())[0]

('GOPR0476_frame_000931',
 {(280, 0): {'c_info_gain': 1047887.5549786069,
   'num_pixels': 494784,
   'certified_pixels': 382023,
   'certified_pos': 355887,
   'boundary_pixels': 35009,
   'nonboundary_pixels': 459775,
   'certified_boundary_pixels': 122400,
   'certified_nonboundary_pixels': 459775,
   'pos_certified_boundary_pixels': 13079,
   'pos_certified_nonboundary_pixels': 342808},
  (280, 1): {'c_info_gain': 816279.1721138097,
   'num_pixels': 494784,
   'certified_pixels': 397402,
   'certified_pos': 371453,
   'boundary_pixels': 35009,
   'nonboundary_pixels': 459775,
   'certified_boundary_pixels': 115939,
   'certified_nonboundary_pixels': 459775,
   'pos_certified_boundary_pixels': 18818,
   'pos_certified_nonboundary_pixels': 352635},
  (280, 2): {'c_info_gain': 569144.9280080372,
   'num_pixels': 494784,
   'certified_pixels': 462816,
   'certified_pos': 455169,
   'boundary_pixels': 35009,
   'nonboundary_pixels': 459775,
   'certified_boundary_pixels': 55471,
   'cer

In [10]:
baseline_dict = {}

baseline_stats = baseline_stats_cs
for baseline_stats, name, in zip([baseline_stats_cs, baseline_stats_acdc], ['cityscapes', 'acdc']):
    baseline_acc_dict = {}
    for h in range(4):
        for image in baseline_stats.keys():
            if h not in baseline_acc_dict:
                baseline_acc_dict[h] = {'acc':[], 'confusion_matrix':np.zeros((20, 20)), 'info_gain': []}
            baseline_acc_dict[h]['acc'].append(baseline_stats[image][h]['acc'])
            baseline_acc_dict[h]['info_gain'].append(baseline_stats[image][h]['info_gain'])
            # print(h, baseline_stats[image][h]['acc'])
            baseline_acc_dict[h]['confusion_matrix'] += baseline_stats[image][h]['confusion_matrix']
    baseline_dict[name] = baseline_acc_dict

baseline_acc_dict = baseline_dict['cityscapes']
for h in range(4):
    
    print(h, np.array(baseline_acc_dict[h]['acc']).mean(), )

0 0.8878591776373131
1 0.9347644859399112
2 0.9467302634674725
3 0.9778790791946548


In [43]:
datasets_dict = {}
for stats, name, in zip([stats_cs, ], ['cityscapes', ]):
    graph_dict = {}
    for image in stats.keys():
        n_h_dict = stats[image]
        for n_h_t in n_h_dict.keys():
            n, h = n_h_t
            
            if n_h_t not in graph_dict:
                graph_dict[n_h_t] = {
                    'num_pixels':0, 'certified_pixels':0, 'c_info_gain':0, 'certified_pos':0, 'boundary_pixels':0, 'nonboundary_pixels':0, 'pos_certified_boundary_pixels':0, 'pos_certified_nonboundary_pixels':0,
                    'num_h_pixels':{}, 'pos_h_pixels':{}
                }
            #print(baseline_stats[image][h]['acc']> stats[image][n_h]['certified_pos']/stats[image][n_h]['num_pixels'])
            graph_dict[n_h_t]['num_pixels'] += stats[image][n_h_t]['num_pixels']
            graph_dict[n_h_t]['certified_pixels'] += stats[image][n_h_t]['certified_pixels']
            graph_dict[n_h_t]['certified_pos'] += stats[image][n_h_t]['certified_pos']
            graph_dict[n_h_t]['c_info_gain'] += stats[image][n_h_t]['c_info_gain']

            if h == 4: 
                for i in range(4):
                    if i not in graph_dict[n_h_t]['num_h_pixels']:
                        graph_dict[n_h_t]['num_h_pixels'][i] = 0
                        graph_dict[n_h_t]['pos_h_pixels'][i] = 0

                    graph_dict[n_h_t]['num_h_pixels'][i] += stats[image][n_h_t]['hierarchical_acc'][i]['num_h_pixels']
                    graph_dict[n_h_t]['pos_h_pixels'][i] += stats[image][n_h_t]['hierarchical_acc'][i]['pos']

                continue
            graph_dict[n_h_t]['boundary_pixels'] += stats[image][n_h_t]['boundary_pixels']
            graph_dict[n_h_t]['nonboundary_pixels'] += stats[image][n_h_t]['nonboundary_pixels']
            graph_dict[n_h_t]['pos_certified_boundary_pixels'] += stats[image][n_h_t]['pos_certified_boundary_pixels']
            graph_dict[n_h_t]['pos_certified_nonboundary_pixels'] += stats[image][n_h_t]['pos_certified_nonboundary_pixels']
    datasets_dict[name] = graph_dict


In [9]:
for dataset in datasets_dict:
    graph_dict = datasets_dict[dataset]
    for n_h in graph_dict.keys():
        print(dataset, n_h, graph_dict[n_h]['c_info_gain']/graph_dict[n_h]['num_pixels']) 

cityscapes (280, 0) 2.706362203844921
cityscapes (280, 1) 1.8999469591315319
cityscapes (280, 2) 1.6428506114569756
cityscapes (280, 3) 0.9866267742827309
cityscapes (280, 4) 2.628437520233256
cityscapes (200, 0) 2.5892190589351403
cityscapes (200, 1) 1.9409720482291244
cityscapes (200, 2) 1.6690371255816925
cityscapes (200, 3) 0.9849273005640011
cityscapes (200, 4) 2.6495872193850234
cityscapes (100, 0) 2.5855809547598936
cityscapes (100, 1) 1.894043370509458
cityscapes (100, 2) 1.6305705750705481
cityscapes (100, 3) 0.9787581173580887
cityscapes (100, 4) 2.5966568172043925
cityscapes (75, 0) 2.592565098900856
cityscapes (75, 1) 1.9139046160984294
cityscapes (75, 2) 1.6563855447295595
cityscapes (75, 3) 0.9795556366768092
cityscapes (75, 4) 2.6246058330667292


In [186]:
for dataset in datasets_dict:
    graph_dict = datasets_dict[dataset]
    for n_h in graph_dict.keys():
        print(dataset, n_h, (graph_dict[n_h]['num_pixels']-graph_dict[n_h]['certified_pixels'])/graph_dict[n_h]['num_pixels']) 

cityscapes (280, 0) 0.09468610295776948
cityscapes (280, 1) 0.05553555605802268
cityscapes (280, 2) 0.045739354215272365
cityscapes (280, 3) 0.019364512558644343
cityscapes (280, 4) 0.040980844122357925
cityscapes (200, 0) 0.04207824775521007
cityscapes (200, 1) 0.033836077736586305
cityscapes (200, 2) 0.030820889128017132
cityscapes (200, 3) 0.019602755289875755
cityscapes (200, 4) 0.03419357540252214
cityscapes (100, 0) 0.04737029957298185
cityscapes (100, 1) 0.0409233426296677
cityscapes (100, 2) 0.03848528379190533
cityscapes (100, 3) 0.026001004038839842
cityscapes (100, 4) 0.04248253912522443
cityscapes (75, 0) 0.03842381974958334
cityscapes (75, 1) 0.035505134807608085
cityscapes (75, 2) 0.03511575865249982
cityscapes (75, 3) 0.023443000159414706
cityscapes (75, 4) 0.03676293265448773
acdc (280, 0) 0.23729064235174724
acdc (280, 1) 0.17101846491419875
acdc (280, 2) 0.12995120252228765
acdc (280, 3) 0.08922611597830622
acdc (280, 4) 0.1232590551000731
acdc (200, 0) 0.126080584993

In [203]:
x = []
ns = list(sorted(list(set(np.array(list(graph_dict.keys())).T[0:][0]))))
hs = [0, 4] #list(sorted(list(set(np.array(list(graph_dict.keys())).T[0:][1]))))
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
i = 1
show_legend = True
for dataset in datasets_dict:
    
    graph_dict = datasets_dict[dataset]
    for h in hs:
        y = []
        for n in ns:
            y.append(graph_dict[(n, h)]['certified_pos']/graph_dict[(n, h)]['num_pixels'])
        text = [str(y_)[0:4] for y_ in y]
        if h == 4:  fig.add_scatter(x=ns, y=y, text=text, row=1, col=i, name=f'AdaptiveSegCertify', line_color='black', line = dict( width=3), showlegend=show_legend,mode="lines+markers+text", textposition="top center" ); continue
        if h == 0:  fig.add_scatter(x=ns, y=y, row=1, text=text, col=i, name=f'SegCertify', line_color=colors[h], showlegend=show_legend, mode="lines+markers+text")

        #fig.add_scatter(x=ns, y=y, name=f'H{h}', line_color=colors[h])
        fig.add_scatter(x=ns, y=[np.array(baseline_dict[dataset][h]['acc']).mean()]*len(ns), name=f'Baseline-H{h}',  row=1, col=i, line_color='red', line = dict( width=2, dash='dash'), showlegend=show_legend, textposition="top center")

    i+=1
    show_legend = False
fig.update_xaxes( title_text=r'Number of Samples (n)', type='category' )
fig.update_yaxes(title_text= f'Accuracy' )
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.show()

fig.write_image("ca.svg")


KeyboardInterrupt: 

In [207]:
x = []
ns = list(sorted(list(set(np.array(list(graph_dict.keys())).T[0:][0]))))
hs = list(sorted(list(set(np.array(list(graph_dict.keys())).T[0:][1]))))
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
i = 1
show_legend = True
for dataset in datasets_dict:
    
    graph_dict = datasets_dict[dataset]
    for h in hs:
        y = []
        for n in ns:
            y.append(graph_dict[(n, h)]['certified_pixels']/graph_dict[(n, h)]['num_pixels'])
        text = [str(y_)[0:4] for y_ in y]
        if h == 4:  fig.add_scatter(x=ns, y=y, text=text, row=1, col=i, name=f'AdaptiveSegCertify', line_color='black', line = dict( width=3), showlegend=show_legend,mode="lines+markers+text", textposition="top center" ); continue
        if h == 0:  fig.add_scatter(x=ns, y=y, row=1, text=text, col=i, name=f'SegCertify', line_color=colors[h], showlegend=show_legend, mode="lines+markers+text")

        fig.add_scatter(x=ns, y=y, name=f'H{h}', line_color=colors[h], showlegend=show_legend)
        #fig.add_scatter(x=ns, y=[np.array(baseline_dict[dataset][h]['acc']).mean()]*len(ns), name=f'Baseline-H{h}',  row=1, col=i, line_color='red', line = dict( width=2, dash='dash'), showlegend=show_legend, textposition="top center")

    i+=1
    show_legend = False
fig.update_xaxes( title_text=r'Number of Samples (n)', type='category' )
fig.update_yaxes(title_text= f'Accuracy' )
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.show()

#fig.write_image("ca.svg")

In [212]:
n = 75
h = 4

x = range(4)

x = [f'H{i}' for i in range(4)]
fig = go.Figure()
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
i =1
for dataset in datasets_dict.keys():
    graph_dict = datasets_dict[dataset]
    y1, y2 = [], []
    for h in range(4):
        if h==2:y2.append(graph_dict[n_h_t]['pos_h_pixels'][h]); y1.append(graph_dict[n_h_t]['num_h_pixels'][h] - graph_dict[n_h_t]['pos_h_pixels'][h]); continue
        
        y1.append(graph_dict[n_h_t]['pos_h_pixels'][h])
        y2.append(graph_dict[n_h_t]['num_h_pixels'][h] - graph_dict[n_h_t]['pos_h_pixels'][h])

    fig.add_bar(x=x, y=np.array(y1), row=1, col=i, name='certified positive', marker_color='lightslategray', showlegend=show_legend)
    fig.add_bar(x=x, y=np.array(y2), row=1, col=i, name='certified negative', marker_color='crimson', showlegend=show_legend)

    show_legend = not show_legend
    i+=1
    fig.update_xaxes( title_text='Hierarchy level', type='category' )
    fig.update_yaxes(title_text= f'Number of certified pixels',)
    fig.update_layout(
        barmode='stack', margin=dict(l=20, r=20, t=20, b=20),
    )

fig.show()
#image = fig.to_image(format="svg")


In [215]:
image = fig.to_image(format="svg")

KeyboardInterrupt: 

In [117]:
graph_dict[n_h_t]['pos_h_pixels'][2]

52518